Mounting Google Drive to get the dataset

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Requirements

Install all needed libraries

In [2]:
!pip install pytube
!pip install youtube_transcript_api
!python3 -m pip install pyLDAvis
!python3 -m spacy download en
!pip install pandas==1.3.1

     |████████████████████████████████| 12.0 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Import all libraries

In [3]:
! wget https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64
! mv ffmpeg.linux64 ~/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1

--2021-12-02 01:07:52--  https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/imageio/imageio-binaries/master/ffmpeg/ffmpeg.linux64 [following]
--2021-12-02 01:07:53--  https://raw.githubusercontent.com/imageio/imageio-binaries/master/ffmpeg/ffmpeg.linux64
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28549024 (27M) [application/octet-stream]
Saving to: ‘ffmpeg.linux64.1’

ffmpeg.linux64.1    100%[===================>]  27.23M   133MB/s    in 0.2s    

2021-12-02 01:07:53 (133 MB/s) - ‘ffmpeg.linux64.1’ saved [28549024/285

In [4]:
from os import listdir
import os.path
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import pyLDAvis
import pyLDAvis.gensim_models 
from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import re
import sys
import urllib.request
import requests
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from moviepy.editor import *
from bs4 import BeautifulSoup  

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# Data Collection 

YouTube class to hold all the video data

In [5]:
PATH = "/content/"
API_KEY = None

In [6]:
# This class holds all the video data
class YoutubeStats:
    def __init__(self, url, id):
        #You can use the youtube data api v3 by uncommenting the following 3 lines and passing an API key to the class 
        #callUrl = f"https://www.googleapis.com/youtube/v3/videos?part=snippet&id={id}&key={API_KEY}"
        #self.respose = requests.get(callUrl)
        #self.data = json.loads(self.respose.text)
        try:
            self.captionData = YouTubeTranscriptApi.get_transcript(id)
        except:
            self.captionData = None
        

        self.url = url
        self.id = id
        self.title = self.getVideoTitle()

    def printData(self):
        print(self.data)

    def getURL(self):
        return self.url
    
    def getID(self):
        return self.id

    # Returns the video title
    def getVideoTitle(self):
        source = requests.get(self.url).text
        soup = BeautifulSoup(source,'html.parser')
        Title = soup.title.text
        Title = Title[:-10] 
        return Title

    # Returns the video description as a string
    def getVideoDescription(self):
        return self.data["items"][0]["snippet"]["description"]

    # Downloads the video from youtube with 360p resolution
    # given the video URL and the name of the file
    def downloadVideo(self):
        path = PATH + 'videos'
        title = reNameTitle(self.title) + ".mp4"
        print("title ........... ", title)
        YouTube(self.url).streams.first().download(output_path=path, filename=title)

    # Coverts video from mp4 to mp3 given the name of
    # the file and the type of the output file eg: mp3, wav
    def convertVideoToSound(self, type="wav"):
        VideoPath = PATH + 'videos/'
        path = PATH + 'audio/'
        title = reNameTitle(self.title)
        video = VideoFileClip(VideoPath + title + '.mp4')
        video.audio.write_audiofile(path + title + '.' + type)

    # Returns the video transcript as a dictionary with times
    def getVideoTranscript(self):
        return self.captionData

    # Return the video transcript as a string
    def getVideoTranscriptString(self):
        transcript = ""
        addNewLine = False
        for caption in self.captionData:
            if addNewLine:
                transcript += str(caption["text"] + "\n")
                addNewLine = False
            else:
                transcript += str(caption["text"] + " ")
                addNewLine = True
        return transcript
    
    def writeToJsonFile(self):
        path = PATH + 'json_scripts'
        i = 0
        title = self.title
        transcript = self.getVideoTranscript()
        print("{0} Writing to json file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}.json", "w") as file:
            json.dump(transcript, file)
    def writeTranscriptFile(self):
        path = "/content/drive/MyDrive/nlp_videos/nlp_transcripts"
        i = 0
        title = self.title
        transcript = self.getVideoTranscriptString()
        print("{0} Writing to transcript file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}_transcript.txt", "w") as file:
            file.write(transcript)


Supporting Functions

In [7]:
# This function create a file for every video
# and it assigns the file name to the title of
# that video. It then writes the title, description
# and the transcript to the file
def writeToVideoFile(youtubeStats):
    path = createDirectory('video_scripts')
    i = 0
    for stats in youtubeStats:
        title = stats.title
        #description = stats.getVideoDescription()
        transcript = stats.getVideoTranscriptString()
        print("{0} Writing to file...".format(i+1))
        i+=1
        with open(path + '/'f"{reNameTitle(title)}.txt", "w") as file:
            file.write("Title\n\n" + title)
            file.write(
                "\n--------------------------------------------------------------------\n")
            #file.write("\nDescription\n\n" + description)
            #file.write(
            #    "\n\n--------------------------------------------------------------------\n")
            file.write("\nTranscript\n\n" + transcript)
            file.write(
                "\n--------------------------------------------------------------------\n")

In [8]:
#download all the videos in the input file
def downloadAllVideos(youtubeStats):
    createDirectory('videos')   
    i = 0
    for stats in youtubeStats:
        print("{0} Downloading...".format(i+1))
        i+=1
        stats.downloadVideo(stats.getURL(), stats.title) 

In [9]:
def convertAllVideosToSound(youtubeStats):
    createDirectory('audio')
    for stats in youtubeStats:
        stats.convertVideoToSound(stats.title, 'wav')


In [10]:
# Helper function to rename the file
def reNameTitle(title):
    title = re.sub('[\W_]+', "_", title)
    return title.lower()

In [11]:
# Helper function to create a directory
def createDirectory(name):
    path = PATH + name
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [12]:
def converYoutubeLinkToId(link):
  return link.split("=", 1)[1].strip()

In [13]:
# Reads a list of URLs from a given file
# and parses them to video IDs.
# Returns a list of URLs and video IDs
def readFile(path):
    videoIds = []
    urls = []
    with open(path, "r") as file:
        for line in file:
            videoIds.append(line.split("=", 1)[1].strip())
            urls.append(line.strip())
    return videoIds, urls

# Data Processing

Data Preprocessing

In [14]:
nlp = spacy.load("en_core_web_sm")

Read my stop words from Google Drive

In [15]:
def get_stop_words_from_drive():
  with open('/content/drive/My Drive/NLP Project/stop_words.txt', "r") as f:
    return f.read().splitlines()

In [16]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

In [17]:
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

In [18]:
def pass_through_pipeline(documents_from_files):
  documents = []
  for doc in documents_from_files:
    documents.append(nlp(doc))

  return documents

In [19]:
def data_preprocessing():
  # My list of stop words
  stop_words = get_stop_words_from_drive()
  
  #Add my stop words to Spacy's stop words
  nlp.Defaults.stop_words.update(stop_words)

  # Iterates over the words in the stop words list and resets the "is_stop" flag.
  for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

  # The add_pipe function appends our functions to the default pipeline.
  nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
  nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

# Results

In [20]:
data_preprocessing()

In [21]:
videoURL = input("Enter the Youtube video link: ")
videoId = converYoutubeLinkToId(videoURL)
youtubeStats = YoutubeStats(videoURL, videoId)
transcript= youtubeStats.getVideoTranscriptString()
createDirectory('transcripts')
youtubeStats.writeTranscriptFile()
documents = pass_through_pipeline([transcript])

Enter the Youtube video link: https://www.youtube.com/watch?v=yZKhfZ25L4o&t
Creation of the directory /content/transcripts failed
1 Writing to transcript file...


In [22]:
import pickle
with open("/content/drive/My Drive/topic_files/words_20000.pickle", 'rb') as file:
  words2 = pickle.load(file)

with open("/content/drive/My Drive/topic_files/corpus_20000.pickle", 'rb') as file:
  corpus2 = pickle.load(file)

with open("/content/drive/My Drive/topic_files/50topics_lda.pickle", 'rb') as file:
  lda2_model2 = pickle.load(file)  

In [23]:

# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(documents)

# Turns each document into a bag of words.
corpus = [words2.doc2bow(doc) for doc in documents]

# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=words2,num_topics=10,)



In [24]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda2_model2, corpus, words2)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.037975  0.133273       1        1  24.183847
47     0.037992  0.131866       2        1   9.877650
6      0.047161  0.087918       3        1   6.742899
39     0.020691  0.127462       4        1   5.377419
0      0.047840  0.026639       5        1   5.225872
36     0.059849  0.096053       6        1   4.891950
46     0.031364  0.055258       7        1   4.109266
48     0.041276 -0.099205       8        1   3.878441
10     0.035066  0.078766       9        1   3.402218
33     0.018642  0.144770      10        1   3.152930
11     0.040573  0.002210      11        1   3.032738
15     0.059592 -0.020759      12        1   2.394534
37     0.040610  0.093514      13        1   2.325916
5      0.044643 -0.043166      14        1   2.153558
26     0.026581  0.057032      15        1   2.080867
24     0.040719  0.064408      16        1   2.004808
3      0.053387 -0.079954      17        1   1.768267
14     0.043665  0.034000      18        1   1.261024
12     0.034428 -0.055747      19        1   1.050173
2     -0.019150  0.038209      20        1   0.980380
20     0.063944 -0.112209      21        1   0.846831
31     0.032219 -0.173484      22        1   0.762946
29    -0.042045  0.024188      23        1   0.734361
13     0.033518 -0.101727      24        1   0.682106
41     0.021344  0.070065      25        1   0.665431
40     0.018722  0.052655      26        1   0.664662
35     0.029306  0.043372      27        1   0.613938
32     0.013704  0.058071      28        1   0.583787
1      0.032249 -0.114088      29        1   0.567566
19    -0.018569  0.009570      30        1   0.558009
21     0.031801 -0.000812      31        1   0.528921
17     0.042862 -0.069845      32        1   0.409519
28    -0.144957 -0.015701      33        1   0.393306
27     0.042413  0.002475      34        1   0.288743
42     0.044745 -0.117446      35        1   0.262595
23     0.002835  0.044208      36        1   0.260987
16     0.028972 -0.061660      37        1   0.256905
30     0.048589 -0.093469      38        1   0.247368
22    -0.055163  0.011950      39        1   0.225464
49     0.016544 -0.048667      40        1   0.148214
7     -0.034743 -0.072318      41        1   0.113664
43    -0.107652  0.034517      42        1   0.106306
18     0.012437 -0.053024      43        1   0.092227
45    -0.021752 -0.031695      44        1   0.050702
44    -0.051081 -0.087657      45        1   0.039026
38    -0.266193  0.022553      46        1   0.000332
34     0.033488  0.001526      47        1   0.000332
8     -0.037332 -0.080688      48        1   0.000332
4     -0.210788 -0.007644      49        1   0.000332
25    -0.302319 -0.005563      50        1   0.000332, topic_info=             Term        Freq       Total Category  logprob  loglift
133         fight  112.000000  112.000000  Default  30.0000  30.0000
276           use   25.000000   25.000000  Default  29.0000  29.0000
73            big   37.000000   37.000000  Default  28.0000  28.0000
175          kind   26.000000   26.000000  Default  27.0000  27.0000
554          year   27.000000   27.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
23961       Rhino    0.000055    0.026934  Topic50  -5.9018   6.4171
15634        dave    0.000052    0.026798  Topic50  -5.9579   6.3660
111467      Foote    0.000052    0.038619  Topic50  -5.9608   5.9977
111468     Sparty    0.000052    0.038619  Topic50  -5.9608   5.9977
111469  Stockport    0.000052    0.038619  Topic50  -5.9608   5.9977

[4048 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
556       5  0.247153      $
556      18  0.247153      $
906       1  0.314323     10
906       2  0.104774     10
906       3  0.104774     10
...     ...       ...    ...
292       1  0.338482    yes
292       2  0.084620 